In [114]:
# This code reads the data into a pandas data frame. The dataset used is an Office Products dataset containing 53,257 reviews   
# retrieved from: http://jmcauley.ucsd.edu/data/amazon/ citation: "Image-based recommendations on styles and 
# substitutes" J. McAuley, C. Targett, J. Shi, A. van den Hengel, SIGIR, 2015

import ctypes, inspect, os, graphlab
from ctypes import wintypes
kernel32 = ctypes.WinDLL('kernel32', use_last_error=True)
kernel32.SetDllDirectoryW.argtypes = (wintypes.LPCWSTR,)
src_dir = os.path.split(inspect.getfile(graphlab))[0]
kernel32.SetDllDirectoryW(src_dir)

from graphlab import SFrame
import pandas as pd

def parse(path):
#  opens a binary file in read or write mode.    
  g = open(path, 'rb')
  for l in g:
    yield eval(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

df = getDF('Office_Products_5.json')



In [100]:
df.head()

,reviewerID,asin,reviewerName,helpful,unixReviewTime,reviewText,overall,reviewTime,summary
0,A32T2H8150OJLU,B00000JBLH,ARH,"[3, 4]",1094169600,"I bought my first HP12C in about 1984 or so, a...",5,"09 3, 2004","A solid performer, and long time friend"
1,A3MAFS04ZABRGO,B00000JBLH,"Let it Be ""Alan""","[7, 9]",1197676800,WHY THIS BELATED REVIEW? I feel very obliged t...,5,"12 15, 2007","Price of GOLD is up, so don't bury the golden ..."
2,A1F1A0QQP2XVH5,B00000JBLH,Mark B,"[3, 3]",1293840000,I have an HP 48GX that has been kicking for mo...,2,"01 1, 2011","Good functionality, but not durable like old HPs"
3,A49R5DBXXQDE5,B00000JBLH,R. D Johnson,"[7, 8]",1145404800,I've started doing more finance stuff recently...,5,"04 19, 2006",One of the last of an almost extinct species
4,A2XRMQA6PJ5ZJ8,B00000JBLH,Roger J. Buffington,"[0, 0]",1375574400,For simple calculations and discounted cash fl...,5,"08 4, 2013",Still the best


In [101]:
df_review_data['word_count'] = graphlab.text_analytics.count_words(df['reviewText'])

ToolkitError: Input sa is not an SArray.

In [71]:
# Limiting the review data to only helpful reviews. The ‘helpful’ column contains values like ‘[x, y]’.
# The first value represents the number of helpful votes, the second represents overall votes. 
# If at least 60% consider the review helpful, we want those reviews.
import numpy as np

df_review_data['helpful_count'] = df_review_data.helpful.apply(lambda x: x[0])
df_review_data['overall_count'] = df_review_data.helpful.apply(lambda x: x[1])
df_review_data['helpful_percent'] = df_review_data['helpful_count'] / df_review_data['overall_count']
df_review_data['helpful_review'] = np.where((df_review_data.helpful_percent >= .6) & (df_review_data.overall_count > 0), "Yes", "No")
helpful_reviews = df_review_data[df_review_data['helpful_review']=='Yes']
unhelpful_reviews = df_review_data[df_review_data['helpful_review']=='No']

In [78]:
Office_products = graphlab.SFrame.read_json('Office_Products_5.json', orient='records')

Unable to parse line "{"reviewerID": "A32T2H8150OJLU", "asin": "B00000JBLH", "reviewerName": "ARH", "helpful": [3, 4], "reviewText": "I bought my first HP12C in about 1984 or so, and it served me faithfully until 2002 when I lost it while travelling.  I searched for another one..."

1 lines failed to parse correctly

Finished parsing file C:\Users\Chauncey\Office_Products_5.json

Parsing completed. Parsed 0 lines in 18.193 secs.

RuntimeError: Input JSON not of expected format

In [102]:
sf = SFrame(data=df)

TypeError: A common type cannot be infered from types float, string.

In [83]:
df_review_data['reviewText'].head(5)

0    I bought my first HP12C in about 1984 or so, a...
1    WHY THIS BELATED REVIEW? I feel very obliged t...
2    I have an HP 48GX that has been kicking for mo...
3    I've started doing more finance stuff recently...
4    For simple calculations and discounted cash fl...
Name: reviewText, dtype: object

In [97]:
sf.head()

Columns:
	None

Rows: 0

Data:
	[]

In [104]:
st = SFrame.read_json('Office_Products_5.json', orient='lines')

Finished parsing file C:\Users\Chauncey\Office_Products_5.json

Parsing completed. Parsed 100 lines in 3.11318 secs.

Read 50429 lines. Lines per second: 40996.9

Finished parsing file C:\Users\Chauncey\Office_Products_5.json

Parsing completed. Parsed 53258 lines in 1.25307 secs.

------------------------------------------------------
Inferred types from first line of file as 
column_type_hints=[dict]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


In [105]:
st.head()

asin,helpful,overall,reviewText,reviewTime,reviewerID,reviewerName
B00000JBLH,"[3, 4]",5.0,"I bought my first HP12Cin about 1984 or so, and ...","09 3, 2004",A32T2H8150OJLU,ARH
B00000JBLH,"[7, 9]",5.0,WHY THIS BELATED REVIEW?I feel very obliged to ...,"12 15, 2007",A3MAFS04ZABRGO,"Let it Be ""Alan"""
B00000JBLH,"[3, 3]",2.0,I have an HP 48GX thathas been kicking for ...,"01 1, 2011",A1F1A0QQP2XVH5,Mark B
B00000JBLH,"[7, 8]",5.0,I've started doing morefinance stuff recently ...,"04 19, 2006",A49R5DBXXQDE5,R. D Johnson
B00000JBLH,"[0, 0]",5.0,For simple calculationsand discounted cash ...,"08 4, 2013",A2XRMQA6PJ5ZJ8,Roger J. Buffington
B00000JBLH,"[10, 12]",5.0,"While I don't have anMBA, it's hard to bel ...","01 23, 2002",A2JFOHC9W629IE,scott_from_dallas
B00000JBLH,"[3, 4]",5.0,I've had an HP 12C eversince they were first ...,"01 17, 2007",A38NELQT98S4H8,W. B. Halper
B00000JBLH,"[0, 0]",5.0,Bought this for my bossbecause he lost his. He ...,"11 14, 2013",AA8M6331NI1EN,ZombieMom
B00000JBLU,"[3, 3]",5.0,"This is a well-designed,simple calculator that ...","12 7, 2010",A25C2M3QF9G7OQ,Comdet
B00000JBLU,"[0, 0]",5.0,"I love this calculator,big numbers and calcu ...","12 2, 2013",A1RTVWTWZSIC94,"Hb ""Black Beauty"""


In [106]:
# building a word count vector for each review 
st['word_count'] = graphlab.text_analytics.count_words(st['reviewText'])

In [125]:
st.head()

asin,helpful,overall,reviewText,reviewTime,reviewerID,reviewerName
B00000JBLH,"[3, 4]",5.0,"I bought my first HP12Cin about 1984 or so, and ...","09 3, 2004",A32T2H8150OJLU,ARH
B00000JBLH,"[7, 9]",5.0,WHY THIS BELATED REVIEW?I feel very obliged to ...,"12 15, 2007",A3MAFS04ZABRGO,"Let it Be ""Alan"""
B00000JBLH,"[3, 3]",2.0,I have an HP 48GX thathas been kicking for ...,"01 1, 2011",A1F1A0QQP2XVH5,Mark B
B00000JBLH,"[7, 8]",5.0,I've started doing morefinance stuff recently ...,"04 19, 2006",A49R5DBXXQDE5,R. D Johnson
B00000JBLH,"[0, 0]",5.0,For simple calculationsand discounted cash ...,"08 4, 2013",A2XRMQA6PJ5ZJ8,Roger J. Buffington
B00000JBLH,"[10, 12]",5.0,"While I don't have anMBA, it's hard to bel ...","01 23, 2002",A2JFOHC9W629IE,scott_from_dallas
B00000JBLH,"[3, 4]",5.0,I've had an HP 12C eversince they were first ...,"01 17, 2007",A38NELQT98S4H8,W. B. Halper
B00000JBLH,"[0, 0]",5.0,Bought this for my bossbecause he lost his. He ...,"11 14, 2013",AA8M6331NI1EN,ZombieMom
B00000JBLU,"[3, 3]",5.0,"This is a well-designed,simple calculator that ...","12 7, 2010",A25C2M3QF9G7OQ,Comdet
B00000JBLU,"[0, 0]",5.0,"I love this calculator,big numbers and calcu ...","12 2, 2013",A1RTVWTWZSIC94,"Hb ""Black Beauty"""


In [109]:
graphlab.canvas.set_target('ipynb')

In [126]:
# Limiting the review data to only helpful reviews. The ‘helpful’ column contains values like ‘[x, y]’.
# The first value represents the number of helpful votes, the second represents overall votes. 
# If at least 60% consider the review helpful, we want those reviews.
import numpy as np

st_dataframe = st.to_dataframe()


In [129]:

print(st_dataframe.head(5))
len(st_dataframe)

         asin helpful  overall  \
0  B00000JBLH  [3, 4]        5   
1  B00000JBLH  [7, 9]        5   
2  B00000JBLH  [3, 3]        2   
3  B00000JBLH  [7, 8]        5   
4  B00000JBLH  [0, 0]        5   

                                          reviewText   reviewTime  \
0  I bought my first HP12C in about 1984 or so, a...   09 3, 2004   
1  WHY THIS BELATED REVIEW? I feel very obliged t...  12 15, 2007   
2  I have an HP 48GX that has been kicking for mo...   01 1, 2011   
3  I've started doing more finance stuff recently...  04 19, 2006   
4  For simple calculations and discounted cash fl...   08 4, 2013   

       reviewerID         reviewerName  \
0  A32T2H8150OJLU                  ARH   
1  A3MAFS04ZABRGO     Let it Be "Alan"   
2  A1F1A0QQP2XVH5               Mark B   
3   A49R5DBXXQDE5         R. D Johnson   
4  A2XRMQA6PJ5ZJ8  Roger J. Buffington   

                                             summary  unixReviewTime  \
0            A solid performer, and long time friend   

53258

In [130]:
import numpy as np
import pandas as pd



st_dataframe['helpful_count'] = st_dataframe.helpful.apply(lambda x: x[0])
st_dataframe['overall_count'] = st_dataframe.helpful.apply(lambda x: x[1])
st_dataframe['helpful_percent'] = st_dataframe['helpful_count'] / st_dataframe['overall_count']
st_dataframe['helpful_review'] = np.where((st_dataframe.helpful_percent >= .6) & (st_dataframe.overall_count > 0), "Yes", "No")
helpful_reviews = st_dataframe[st_dataframe['helpful_review']=='Yes']
unhelpful_reviews = st_dataframe[st_dataframe['helpful_review']=='No']



In [131]:
st_dataframe.head()

,asin,helpful,overall,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime,word_count,helpful_count,overall_count,helpful_percent,helpful_review
0,B00000JBLH,"[3, 4]",5,"I bought my first HP12C in about 1984 or so, a...","09 3, 2004",A32T2H8150OJLU,ARH,"A solid performer, and long time friend",1094169600,"{u'area.': 1, u'hand,': 1, u'over': 1, u'move'...",3,4,0.750000,Yes
1,B00000JBLH,"[7, 9]",5,WHY THIS BELATED REVIEW? I feel very obliged t...,"12 15, 2007",A3MAFS04ZABRGO,"Let it Be ""Alan""","Price of GOLD is up, so don't bury the golden ...",1197676800,"{u'all': 3, u'ever.': 1, u'four': 1, u'desktop...",7,9,0.777778,Yes
2,B00000JBLH,"[3, 3]",2,I have an HP 48GX that has been kicking for mo...,"01 1, 2011",A1F1A0QQP2XVH5,Mark B,"Good functionality, but not durable like old HPs",1293840000,"{u'and': 4, u'12c': 1, u'old': 1, u'classic': ...",3,3,1.000000,Yes
3,B00000JBLH,"[7, 8]",5,I've started doing more finance stuff recently...,"04 19, 2006",A49R5DBXXQDE5,R. D Johnson,One of the last of an almost extinct species,1145404800,"{u'12c': 2, u'stamped': 1, u'cellphones,': 1, ...",7,8,0.875000,Yes
4,B00000JBLH,"[0, 0]",5,For simple calculations and discounted cash fl...,"08 4, 2013",A2XRMQA6PJ5ZJ8,Roger J. Buffington,Still the best,1375574400,"{u'office': 1, u'flows,': 1, u'years': 1, u'co...",0,0,NaN,No


In [132]:
Office_Products_sframe = graphlab.SFrame(st_dataframe)

In [133]:
Office_Products_sframe.head()

asin,helpful,overall,reviewText,reviewTime,reviewerID
B00000JBLH,"[3.0, 4.0]",5.0,"I bought my first HP12Cin about 1984 or so, and ...","09 3, 2004",A32T2H8150OJLU
B00000JBLH,"[7.0, 9.0]",5.0,WHY THIS BELATED REVIEW?I feel very obliged to ...,"12 15, 2007",A3MAFS04ZABRGO
B00000JBLH,"[3.0, 3.0]",2.0,I have an HP 48GX thathas been kicking for ...,"01 1, 2011",A1F1A0QQP2XVH5
B00000JBLH,"[7.0, 8.0]",5.0,I've started doing morefinance stuff recently ...,"04 19, 2006",A49R5DBXXQDE5
B00000JBLH,"[0.0, 0.0]",5.0,For simple calculationsand discounted cash ...,"08 4, 2013",A2XRMQA6PJ5ZJ8
B00000JBLH,"[10.0, 12.0]",5.0,"While I don't have anMBA, it's hard to bel ...","01 23, 2002",A2JFOHC9W629IE
B00000JBLH,"[3.0, 4.0]",5.0,I've had an HP 12C eversince they were first ...,"01 17, 2007",A38NELQT98S4H8
B00000JBLH,"[0.0, 0.0]",5.0,Bought this for my bossbecause he lost his. He ...,"11 14, 2013",AA8M6331NI1EN
B00000JBLU,"[3.0, 3.0]",5.0,"This is a well-designed,simple calculator that ...","12 7, 2010",A25C2M3QF9G7OQ
B00000JBLU,"[0.0, 0.0]",5.0,"I love this calculator,big numbers and calcu ...","12 2, 2013",A1RTVWTWZSIC94


In [136]:
# Building the sentiment classifier 
train_data, test_data = Office_Products_sframe.random_split(.8, seed=0)